# Test esatto di Fisher

- Con il seguente esempio, mostriamo come applicare il test esatto di Fisher per verificare l'indipendenza tra due variabili categoriche, ciascuna avente solo due livelli --> tabella 2 x 2.
- **Steps**:
    
    - Racconto dell'aneddoto del tè e latte di Ronald Fisher
    - Conteggi osservati e conteggi attesi sotto l'ipotesi di indipendenza
    - Rapporto tra gli odds (statistica test) e *p*-value esatto
    - Interpretazione
    - Determinazione del *mid*-*p*-value per essere meno conservativi.

In [1]:
%pylab inline
from scipy import stats

Populating the interactive namespace from numpy and matplotlib


## L'aneddoto del tè e latte di Ronald Fisher

Muriel Bristol asserì di avere la capacità di dire, bevendo una tazza di tè e latte, se fosse stato versato prima il latte oppure il tè. Fisher, incuriosito da ciò, le sottopose in modo randomizzato 8 tazze di tè e latte, in 4 delle quali era stato versato prima il latte, mentre nelle 4 restanti era stato versato prima il tè.  
La signora Bristol sbagliò a riconoscere una tazza in cui era stato versato prima il latte e una tazza in cui era stato versato prima il tè.  
Possiamo quindi individuare due variabili dicotomiche (entrambe con modalità Sì/No): **versatoLatte** ("Il latte è versato per primo") e **scommetteLatte** ("Il latte viene riconosciuto come primo ad essere versato").  
Se la prima variabile è posta sulle righe e la seconda è posta sulle colonne, la tabella di contingenza (conteggi **osservati**) è la seguente:

In [2]:
esempio_Fisher = array([[3.0, 1.0], [1.0, 3.0]])
esempio_Fisher

array([[ 3.,  1.],
       [ 1.,  3.]])

In caso di **indipendenza** di una variabile dall'altra, siamo nella situazione in cui la signora Bristol non sappia davvero riconoscere quando venga versato prima il latte oppure il tè. Dunque la tabella dei conteggi **attesi** è la seguente:

In [3]:
array([[2.0, 2.0], [2.0, 2.0]])

array([[ 2.,  2.],
       [ 2.,  2.]])

In caso di **indipendenza**, gli *odds* di riconoscere che sia stato versato prima il latte quando è stato versato prima il latte e gli odds di riconoscere che sia stato versato prima il latte quando invece è stato versato prima il tè sono gli stessi, dunque il loro rapporto sotto l'ipotesi nulla è uguale a 1.

In [4]:
scommetteLatte_versatoLatte = esempio_Fisher[0,0]
scommetteLatte_versatoLatte

3.0

In [5]:
marginali_riga, marginali_colonna = np.sum(esempio_Fisher, axis= 0), np.sum(esempio_Fisher, axis= 1)
tazze_totali, tazze_versatoLatte, tazze_scommetteLatte = np.sum(esempio_Fisher), marginali_riga[0], marginali_colonna[0]
tazze_totali, tazze_versatoLatte, tazze_scommetteLatte

(8.0, 4.0, 4.0)

Mantenendo fissi questi marginali, le altre situazioni tanto estreme o più estreme (sotto ipotesi di indipendenza) rispetto a quella osservata si hanno quando il conteggio per **scommetteLatte_versatoLatte** è uguale a 0, 1, oppure 4.

### Rapporto tra gli odds per la tabella osservata

Calcoliamo il rapporto tra gli *odds* sopra descritti. L'obiettivo sarà quello di decidere se tale rapporto sia o no lontano (maggiore o minore che sia) dal valore 1, ossia il valore atteso sotto ipotesi di indipendenza.

In [6]:
prob_scommetteLatte_versatoLatte = scommetteLatte_versatoLatte/tazze_versatoLatte
odds_scommetteLatte_versatoLatte = prob_scommetteLatte_versatoLatte/(1 - prob_scommetteLatte_versatoLatte) 

prob_scommetteLatte_versatoTe = (tazze_scommetteLatte - scommetteLatte_versatoLatte)/(tazze_totali - tazze_versatoLatte) 
odds_scommetteLatte_versatoTe = prob_scommetteLatte_versatoTe/(1 - prob_scommetteLatte_versatoTe)

rapporto_odds = odds_scommetteLatte_versatoLatte/odds_scommetteLatte_versatoTe
rapporto_odds

9.0

Questo valore indica che, secondo i conteggi osservati, risulta 9 volte più probabile che la signora Bristol riconosca che il latte sia il primo ad essere versato quando il latte viene effettivamente versato per primo rispetto a quando il latte viene versato per secondo. Per capire se un tale valore del rapporto tra gli *odds* sia sufficiente per dire che la signora Bristol abbia davvero l'abilità che dice di avere, dobbiamo effettuare un test (**test esatto di Fisher**), il cui valore osservato della statistica test è proprio questo rapporto tra gli *odds* appena calcolato.

### Calcolo del *p*-value esatto del test 

Il *p*-value esatto è calcolato considerando la somma della probabilità di osservare, sotto l'ipotesi nulla di indipendenza, situazioni tanto estreme o più estreme di quella osservata:

In [7]:
vc_iperg = stats.hypergeom(tazze_totali, tazze_versatoLatte, tazze_scommetteLatte)
pmf_situaz_cosi_estreme = vc_iperg.pmf(scommetteLatte_versatoLatte) + vc_iperg.pmf(1)
pmf_situaz_piu_estreme = vc_iperg.pmf(0) + vc_iperg.pmf(4)
p_value_esatto = pmf_situaz_cosi_estreme + pmf_situaz_piu_estreme
p_value_esatto

0.48571428571428527

### Il test dà dunque il seguente esito in termini di statistica test (rapporto tra gli *odds*) e *p*-value

In [8]:
rapporto_odds, p_value_esatto

(9.0, 0.48571428571428527)

### Il test è disponibile anche in scipy --> stats:

In [9]:
stats.fisher_exact(esempio_Fisher)

(9.0, 0.48571428571428527)

### Interpretazione

Il test non riesce a rifiutare l'ipotesi nulla di indipendenza tra le due variabili dicotomiche e dunque non possiamo affermare che la signora Bristol abbia davvero l'abilità che dice di avere.

I problemi sono:
1. Troppo pochi conteggi implicano una potenza del test molto bassa;
2. il *p*-value esatto del test di Fisher è conservativo.

Per ovviare al secondo punto, possiamo ricorrere al *mid-p*-value, che è da molti considerato migliore di quello esatto in quanto meno conservativo:

In [10]:
0.5 * pmf_situaz_cosi_estreme + pmf_situaz_piu_estreme

0.2571428571428569